In [1]:
import sys
import os
import dbfread
from dbfread import DBF
import dbf
from glob import glob 
import pandas as pd
import csv
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
import plotly as py
from plotly import graph_objs as go
from plotly.subplots import make_subplots
import seaborn as sns
from datetime import datetime
import time
from collections import Counter
from tqdm import tqdm
import plotly.express as px
from datetime import datetime

import matplotlib.font_manager as fm
# font_list = fm.findSystemFonts(fontpaths=None, fontext = 'ttf')

from matplotlib import font_manager, rc
# font_path = "C:/Windows/Fonts/NGULIM.TTF"
# font = font_manager.FontProperties(fname=font_path).get_name()
# rc('font', family=font)
rc('font', family='AppleGothic')

import pickle

In [2]:
import os
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from tensorflow.keras import layers, optimizers, models
from tensorflow.keras.layers import BatchNormalization, Bidirectional, Activation 
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras.models import load_model
import tensorflow as tf

import pandas as pd
import numpy as np
import pickle
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import StratifiedKFold
# from sklearn.preprocessing import StandardScaler

In [3]:
#os.chdir('../dataset')
dir_data = os.getcwd() #현재 작업 디렉토리를 가져와 변수에 저장
filename = 'lab_DATASET_v16.pickle' 
#audio_filename = 'lab_DATASET_audio_new_v1.pickle'

with open(os.path.join(dir_data, filename), 'rb') as f:
  DATASET = pickle.load(f)
# with open(os.path.join(dir_data, audio_filename), 'rb') as f:
#   audio_DATASET = pickle.load(f)

In [4]:
# pnum = [1,2,3,4,8,10,12,13,14,16,18,19,20,21,22,23,25,26,27]
# for p in pnum:
#     audio_DATASET[str(p)]['data']['audio_worker'] = audio_DATASET[str(p)]['data']['audio_worker'].drop(['mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13'], axis=1)
#     audio_DATASET[str(p)]['data']['audio_customer'] = audio_DATASET[str(p)]['data']['audio_customer'].drop(['mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13'], axis=1)

In [5]:
# pnum = [1,2,3,4,8,9,10,12,13,14,15,16,18,19,20,21,22,23,25,26,27] # 5: 임산부 / 6: voice없음 / 7: 라벨없음 / 11,17: E4이상 / 24: 임산부
# pnum = [1,2,3,4,8,10,12,13,14,15,16,18,19,20,21,22,23,25,26,27] # 5: 임산부 / 6: voice없음 / 7: 라벨없음 / 9: 웃기다고함 / 11,17: E4이상 / 24: 임산부
#pnum = [1,2,3,4,8,10,12,13,14,16,18,19,20,21,22,23,25,26,27] # 5: 임산부 / 6: voice없음 / 7: 라벨없음 / 9: 웃기다고함 / 11,17: E4이상 / 24: 임산부 / 15: 높은 PSS 점수
pnum = [1]
print(len(pnum))

1


# Load Data and Extract Features

In [6]:
pd.options.mode.chained_assignment = None

from model_utils import eda_mean, eda_count_peaks, eeg_mean_bandpower, temp_mean, signal_interp, label_bin, acc_mag_mean, ecg_hrv, ecg_hr, max_audio, min_audio, std_audio, mean_audio
# import model_utils

# EEG 샘플링 레이트와 레이블 이름 지
eeg_sampling_rate = 256
label_name = 'suppress'

# 다양한 윈도우 및 슬라이딩 값들
# window = 10 
physi_window = 10
audio_window = 5
acc_window = 5
label_window = 5
slide = 1

# # window = 60
# physi_window = 60
# audio_window = 5
# acc_window = 5
# label_window = 5
# slide = 1

call_label = False
call_three_label = False

# 데이터를 저장할 데이터프레임들을 초기화합니다.
EDA_MEAN = pd.DataFrame()
EDA_PEAK = pd.DataFrame()
EEG_MEAN_BANDPOWER = pd.DataFrame()
TEMP_MEAN = pd.DataFrame()
ACC_MEAN = pd.DataFrame()
ECG_HRV = pd.DataFrame()
ECG_HR = pd.DataFrame()
AUDIO_CUSTOMER = pd.DataFrame()
AUDIO_WORKER = pd.DataFrame()
LABEL = pd.DataFrame()

PNUM = pd.DataFrame()

for i in pnum:
    eda_data = DATASET[str(i)]['data']['e4_eda']
    eeg_data = DATASET[str(i)]['data']['muse']
    temp_data = DATASET[str(i)]['data']['e4_temp']
    acc_data = DATASET[str(i)]['data']['e4_acc']
    ecg_rr_data = DATASET[str(i)]['data']['polar_rr']
    ecg_hr_data = DATASET[str(i)]['data']['polar_hr']
    #audio_customer_data = audio_DATASET[str(i)]['data']['audio_customer']
    #audio_customer_data['voiced_flag'] = audio_customer_data['voiced_flag'].astype(int)
    #audio_worker_data = audio_DATASET[str(i)]['data']['audio_worker']
    #audio_worker_data['voiced_flag'] = audio_worker_data['voiced_flag'].astype(int)

    label_data = DATASET[str(i)]['labels'][label_name]
    

    # interpolation
    eeg_data = signal_interp(eeg_data, 256)

    # Data crop
    t_eda = eda_data['Timestamp'].tolist()
    t_eeg = eeg_data['Timestamp'].tolist()
    t_temp = temp_data['Timestamp'].tolist()
    t_acc = acc_data['Timestamp'].tolist()
    t_ecg_rr = ecg_rr_data['Timestamp'].tolist()
    t_ecg_hr = ecg_hr_data['Timestamp'].tolist()
    #t_audio_customer = audio_customer_data['time'].tolist()
    #t_audio_worker = audio_worker_data['time'].tolist()
    t_label = label_data['Timestamp'].tolist()

    max_start = max(t_eda[0], t_eeg[0], t_temp[0], t_acc[0], t_ecg_rr[0], t_ecg_hr[0], 
                    #t_audio_customer[0], t_audio_worker[0], 
                    t_label[0])
    min_end = min(t_eda[-1], t_eeg[-1], t_temp[-1], t_acc[-1], t_ecg_rr[-1], t_ecg_hr[-1], 
                  #t_audio_customer[-1], t_audio_worker[-1], 
                  t_label[-1])

    
    print(max_start, ' ' ,min_end)

    eda_data = eda_data.loc[(eda_data['Timestamp']>=max_start) & (eda_data['Timestamp']<=min_end)]
    eeg_data = eeg_data.loc[(eeg_data['Timestamp']>=max_start) & (eeg_data['Timestamp']<=min_end)] 
    temp_data = temp_data.loc[(temp_data['Timestamp']>=max_start) & (temp_data['Timestamp']<=min_end)]
    acc_data = acc_data.loc[(acc_data['Timestamp']>=max_start) & (acc_data['Timestamp']<=min_end)]
    ecg_rr_data = ecg_rr_data.loc[(ecg_rr_data['Timestamp']>=max_start) & (ecg_rr_data['Timestamp']<=min_end)]
    ecg_hr_data = ecg_hr_data.loc[(ecg_hr_data['Timestamp']>=max_start) & (ecg_hr_data['Timestamp']<=min_end)]
    #audio_customer_data = audio_customer_data.loc[(audio_customer_data['time']>=max_start) & (audio_customer_data['time']<=min_end)]
    #audio_worker_data = audio_worker_data.loc[(audio_worker_data['time']>=max_start) & (audio_worker_data['time']<=min_end)]
    label_data = label_data.loc[(label_data['Timestamp']>=max_start) & (label_data['Timestamp']<=min_end)]        

    # eda_mean
    out_eda_mean = eda_mean(eda_data, physi_window, slide)

    # eda_count_peaks
    out_eda_peak = eda_count_peaks(eda_data, physi_window, slide)

    # eeg_mean_bandpower
    out_eeg_meanbp = eeg_mean_bandpower(eeg_data, sampling_rate=256, window=physi_window, slide=slide, show=False)

    # temp_mean
    out_temp_mean = temp_mean(temp_data, physi_window, slide)
    
    # acc_mean
    out_acc_mag_mean = acc_mag_mean(acc_data, acc_window, slide)

    # ecg_hrv
    out_ecg_hrv = ecg_hrv(ecg_rr_data, physi_window, slide)

    # ecg_hr
    out_ecg_hr = ecg_hr(ecg_hr_data, physi_window, slide)

    # audio_customer
    #out_audio_customer_mean, out_audio_customer_min, out_audio_customer_max, out_audio_customer_std = mean_audio(audio_customer_data, audio_window, slide), min_audio(audio_customer_data, audio_window, slide), max_audio(audio_customer_data, audio_window, slide), std_audio(audio_customer_data, audio_window, slide)

    # audio_worker
    #out_audio_worker_mean, out_audio_worker_min, out_audio_worker_max, out_audio_worker_std = mean_audio(audio_worker_data, audio_window, slide), min_audio(audio_worker_data, audio_window, slide), max_audio(audio_worker_data, audio_window, slide), std_audio(audio_worker_data, audio_window, slide)

    # label binning
    label = label_bin(label_data, label_name, label_window, slide)
    label['pnum'] = i

    print('pnum: ', i)
    print('EDA length: ', len(out_eda_mean))
    print('EEG length: ', len(out_eeg_meanbp))
    print('TEMP length: ', len(out_temp_mean))
    print('ACC length: ', len(out_acc_mag_mean))
    print('ECG HRV length: ', len(out_ecg_hrv))
    print('ECG HR length: ', len(out_ecg_hr))
    #print('AUDIO customer length: ', len(out_audio_customer_mean))
    #print('AUDIO worker length: ', len(out_audio_worker_mean))
    print('LABEL length: ', len(label))


    min_len = min(len(out_eda_mean), len(out_eeg_meanbp), len(out_temp_mean), len(out_acc_mag_mean), len(out_ecg_hrv), len(out_ecg_hr),
                  #len(out_audio_customer_mean), len(out_audio_customer_min), len(out_audio_customer_max), len(out_audio_customer_std),
                  #len(out_audio_worker_mean), len(out_audio_worker_min), len(out_audio_worker_max), len(out_audio_worker_std), 
                  len(label))

    EDA_MEAN = pd.concat([EDA_MEAN, out_eda_mean.iloc[0:min_len]])
    EDA_PEAK = pd.concat([EDA_PEAK, out_eda_peak.iloc[0:min_len]])
    EEG_MEAN_BANDPOWER = pd.concat([EEG_MEAN_BANDPOWER, out_eeg_meanbp.iloc[0:min_len]])
    TEMP_MEAN = pd.concat([TEMP_MEAN, out_temp_mean.iloc[0:min_len]])
    ACC_MEAN = pd.concat([ACC_MEAN, out_acc_mag_mean.iloc[0:min_len]])
    ECG_HRV = pd.concat([ECG_HRV, out_ecg_hrv.iloc[0:min_len]])
    ECG_HR = pd.concat([ECG_HR, out_ecg_hr.iloc[0:min_len]])
    # AUDIO_CUSTOMER_add = pd.concat([out_audio_customer_max.iloc[0:min_len], out_audio_customer_min.iloc[0:min_len], out_audio_customer_mean.iloc[0:min_len], out_audio_customer_std.iloc[0:min_len]], axis=1)
    # AUDIO_CUSTOMER = pd.concat([AUDIO_CUSTOMER, AUDIO_CUSTOMER_add])
    # AUDIO_WORKER_add = pd.concat([out_audio_worker_mean.iloc[0:min_len], out_audio_worker_min.iloc[0:min_len], out_audio_worker_max.iloc[0:min_len], out_audio_worker_std.iloc[0:min_len]], axis=1)
    # AUDIO_WORKER = pd.concat([AUDIO_WORKER, AUDIO_WORKER_add])
    LABEL = pd.concat([LABEL, label.iloc[0:min_len]])

    

#AUDIO_CUSTOMER = AUDIO_CUSTOMER.add_prefix('customer_')
#AUDIO_WORKER = AUDIO_WORKER.add_prefix('worker_')

print(len(EDA_MEAN))
print(len(EDA_PEAK))
print(len(EEG_MEAN_BANDPOWER))
print(len(TEMP_MEAN))
print(len(ACC_MEAN))
print(len(ECG_HRV))
print(len(ECG_HR))
#print(len(AUDIO_CUSTOMER))
#print(len(AUDIO_WORKER))
print(len(LABEL))

1671068339984   1671069492344
pnum:  1
EDA length:  1143
EEG length:  1143
TEMP length:  1143
ACC length:  1148
ECG HRV length:  1142
ECG HR length:  1141
LABEL length:  1148
1141
1141
1141
1141
1141
1141
1141
1141


### Call label (three-label)

In [7]:
call_three_label = True


if call_three_label: 
    call_three_label = []

    for p in pnum: 
        marker = DATASET[str(p)]['data']['marker']
        label_p = LABEL.loc[LABEL['pnum']==p]

        label_p['call_three_label'] = np.nan

        c1_s = marker['Timestamp'][marker['session']=='c1_start']
        c1_e = marker['Timestamp'][marker['session']=='c1_end']
        c2_s = marker['Timestamp'][marker['session']=='c2_start']
        c2_e = marker['Timestamp'][marker['session']=='c2_end']
        c3_s = marker['Timestamp'][marker['session']=='c3_start']
        c3_e = marker['Timestamp'][marker['session']=='c3_end']

        if p%2 == 1:
            label_p.loc[(label_p['Timestamp'] >= c1_s.values[0]) & (label_p['Timestamp'] <= c1_e.values[0]), 'call_three_label'] = 0
            label_p.loc[(label_p['Timestamp'] >= c2_s.values[0]) & (label_p['Timestamp'] <= c2_e.values[0]), 'call_three_label'] = 1
            label_p.loc[(label_p['Timestamp'] >= c3_s.values[0]) & (label_p['Timestamp'] <= c3_e.values[0]), 'call_three_label'] = 2
        elif p%2 == 0: 
            label_p.loc[(label_p['Timestamp'] >= c1_s.values[0]) & (label_p['Timestamp'] <= c1_e.values[0]), 'call_three_label'] = 0
            label_p.loc[(label_p['Timestamp'] >= c2_s.values[0]) & (label_p['Timestamp'] <= c2_e.values[0]), 'call_three_label'] = 2
            label_p.loc[(label_p['Timestamp'] >= c3_s.values[0]) & (label_p['Timestamp'] <= c3_e.values[0]), 'call_three_label'] = 1


        call_three_label.extend(label_p['call_three_label'])

    LABEL['call_three_label'] = call_three_label


### Call label (binary)

In [8]:
call_label = True

if call_label:
    call_label = [] 

    for p in pnum:
        marker = DATASET[str(p)]['data']['marker']
        label_p = LABEL.loc[LABEL['pnum']==p]

        # LABEL = LABEL.reset_index(drop=True)
        # marker['Timestamp'] = marker['Timestamp'].reset_index(drop=True)

        label_p['call_label'] = np.nan

        c1_s = marker['Timestamp'][marker['session']=='c1_start']
        c1_e = marker['Timestamp'][marker['session']=='c1_end']
        c2_s = marker['Timestamp'][marker['session']=='c2_start']
        c2_e = marker['Timestamp'][marker['session']=='c2_end']
        c3_s = marker['Timestamp'][marker['session']=='c3_start']
        c3_e = marker['Timestamp'][marker['session']=='c3_end']

        label_p.loc[(label_p['Timestamp'] >= c1_s.values[0]) & (label_p['Timestamp'] <= c1_e.values[0]), 'call_label'] = 0
        label_p.loc[(label_p['Timestamp'] >= c2_s.values[0]) & (label_p['Timestamp'] <= c2_e.values[0]), 'call_label'] = 1
        label_p.loc[(label_p['Timestamp'] >= c3_s.values[0]) & (label_p['Timestamp'] <= c3_e.values[0]), 'call_label'] = 1

        call_label.extend(label_p['call_label'])

    LABEL['call_label'] = call_label

## Combine features

In [9]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# # Audio features normalization
# AUDIO_CUSTOMER = pd.DataFrame(scaler.fit_transform(AUDIO_CUSTOMER), columns=AUDIO_CUSTOMER.columns)
# AUDIO_WORKER = pd.DataFrame(scaler.fit_transform(AUDIO_WORKER), columns=AUDIO_WORKER.columns)

# # E: 데이터 다시 Load 안해도 되게 원본데이터 보존하려고 temp_ 붙여서 feature combining 하는 것으로 변경
# temp_AUDIO_CUSTOMER = AUDIO_CUSTOMER.drop(columns='customer_Timestamp').reset_index(drop=True)
# temp_AUDIO_WORKER = AUDIO_WORKER.drop(columns='worker_Timestamp').reset_index(drop=True)

In [10]:
# updated features (230) ==> 228개 같아요 (Pnum, suppress, binarized 제외)

#temp_AUDIO_CUSTOMER = AUDIO_CUSTOMER.drop(columns='customer_Timestamp').reset_index(drop=True)
#temp_AUDIO_WORKER = AUDIO_WORKER.drop(columns='worker_Timestamp').reset_index(drop=True)

temp_EDA_MEAN = EDA_MEAN[EDA_MEAN.columns.drop('Timestamp')].reset_index(drop=True)
temp_EDA_PEAK = EDA_PEAK[EDA_PEAK.columns.drop('Timestamp')].reset_index(drop=True)
temp_EEG_MEAN_BANDPOWER = EEG_MEAN_BANDPOWER[EEG_MEAN_BANDPOWER.columns.drop('Timestamp')].reset_index(drop=True)
temp_TEMP_MEAN = TEMP_MEAN[TEMP_MEAN.columns.drop('Timestamp')].reset_index(drop=True)
temp_ACC_MEAN = ACC_MEAN[ACC_MEAN.columns.drop('Timestamp')].reset_index(drop=True)
temp_ECG_HRV = ECG_HRV[ECG_HRV.columns.drop(['Timestamp', 'Datetime'])].reset_index(drop=True)
temp_ECG_HR = ECG_HR[ECG_HR.columns.drop(['Timestamp', 'Datetime'])].reset_index(drop=True)


# old features (210)
# temp_EDA_MEAN = EDA_MEAN[['phasic_mean', 'tonic_mean']].reset_index(drop=True)
# temp_EDA_PEAK = EDA_PEAK[EDA_PEAK.columns.drop('Timestamp')].reset_index(drop=True)
# temp_EEG_MEAN_BANDPOWER = EEG_MEAN_BANDPOWER[EEG_MEAN_BANDPOWER.columns.drop('Timestamp')].reset_index(drop=True)
# temp_TEMP_MEAN = TEMP_MEAN[['temp_mean']].reset_index(drop=True)
# temp_ACC_MEAN = ACC_MEAN[['acc_magnitude']].reset_index(drop=True)
# temp_ECG_HRV = ECG_HRV[ECG_HRV.columns.drop(['Timestamp', 'Datetime'])].reset_index(drop=True)
# temp_ECG_HR = ECG_HR[ECG_HR.columns.drop(['Timestamp', 'Datetime'])].reset_index(drop=True)


# if call_label: 
#     temp_LABEL = LABEL[['call_label', 'pnum']].reset_index(drop=True)
# elif call_three_label:
#     temp_LABEL = LABEL[['call_three_label', 'pnum']].reset_index(drop=True)
# else:
#     temp_LABEL = LABEL[[label_name, 'pnum']].reset_index(drop=True)

temp_LABEL = LABEL[['call_label', 'call_three_label', label_name, 'pnum']].reset_index(drop=True)

# (1) Only situational cues
# result = pd.concat([temp_AUDIO_CUSTOMER, temp_LABEL], axis=1)

# (2) Only required emotion response 
# result = pd.concat([temp_AUDIO_WORKER, temp_LABEL], axis=1)

# (3) Only Not-required emotion response
# result = pd.concat([temp_ACC_MEAN, temp_EDA_MEAN, temp_EDA_PEAK, temp_EEG_MEAN_BANDPOWER, temp_TEMP_MEAN, temp_ECG_HRV, temp_ECG_HR, temp_LABEL], axis=1)

# (4) Situational cues + Required emotion response
# result = pd.concat([temp_AUDIO_CUSTOMER, temp_AUDIO_WORKER, temp_LABEL], axis=1)

# (5) Required emotion response + not-required emotion response
# result = pd.concat([temp_AUDIO_WORKER, temp_ACC_MEAN, temp_EDA_MEAN, temp_EDA_PEAK, temp_EEG_MEAN_BANDPOWER, temp_TEMP_MEAN, temp_ECG_HRV, temp_ECG_HR, temp_LABEL], axis=1)

# (6) Situational cues + not_required emotion response
# result = pd.concat([temp_AUDIO_CUSTOMER, temp_ACC_MEAN, temp_EDA_MEAN, temp_EDA_PEAK, temp_EEG_MEAN_BANDPOWER, temp_TEMP_MEAN, temp_ECG_HRV, temp_ECG_HR, temp_LABEL], axis=1)

# (7) 고객, 상담사, Physiological - 모두 포함
result = pd.concat([
    #temp_AUDIO_CUSTOMER, temp_AUDIO_WORKER, 
    temp_EDA_MEAN, temp_EDA_PEAK, temp_EEG_MEAN_BANDPOWER, temp_TEMP_MEAN, temp_ACC_MEAN, temp_ECG_HRV, temp_ECG_HR, temp_LABEL], axis=1)


# result_nona = result.dropna().reset_index(drop=True)

In [11]:
result_nona = result.dropna().reset_index(drop=True)
result_nona

,EDA_mean,EDA_std,EDA_min,EDA_max,phasic_mean,phasic_std,phasic_min,phasic_max,tonic_mean,tonic_std,...,bpm,sdnn,rmssd,pnn50,hr_mean,hr_std,call_label,call_three_label,suppress,pnum
0,-3.134496,-1.111817,-3.153508,-2.949526,-0.968169,-1.047428,-0.638902,-1.107927,-3.343690,-0.358559,...,1.132466,1.313648,-0.998296,0,0.874645,3.009427,0.0,0.0,0.000000,1
1,-3.132410,-1.105120,-3.153508,-2.949526,-0.984024,-1.052600,-0.526180,-1.107927,-3.333681,-0.186736,...,1.335617,1.017973,-0.999339,0,0.966540,3.170947,0.0,0.0,0.000000,1
2,-3.134362,-1.111576,-3.153508,-2.949526,-1.032241,-1.117802,-0.568930,-1.136465,-3.311891,-0.005586,...,1.502637,0.507330,-1.148316,0,1.136715,3.009427,0.0,0.0,0.000000,1
3,-3.127229,-1.108366,-3.153508,-2.949526,-1.071664,-1.187751,-0.594925,-1.207510,-3.284011,0.047605,...,1.699168,-0.112358,-1.140137,0,1.417503,2.041181,0.0,0.0,0.000000,1
4,-3.107917,-1.095486,-3.147626,-2.934134,-1.095263,-1.237234,-0.594925,-1.264495,-3.254785,-0.037594,...,1.773269,-0.593977,-1.338665,0,1.548538,1.267529,0.0,0.0,0.583333,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
747,-1.225691,-0.064179,-1.355709,-1.123805,-0.160755,0.044568,-0.632897,-0.145487,-1.376630,-0.357067,...,-1.928069,-0.066790,1.113697,0,-1.596295,0.101678,1.0,2.0,19.000000,1
748,-1.185856,-0.221338,-1.355709,-1.123805,-0.115465,-0.034660,-0.632897,-0.145487,-1.350067,-0.038117,...,-1.974528,-0.264534,1.051256,0,-1.671172,-0.205648,1.0,2.0,19.000000,1
749,-1.138687,-0.474454,-1.305712,-1.123805,-0.071912,-0.121642,-0.387239,-0.145487,-1.314656,0.254449,...,-1.902033,0.012139,1.138072,0,-1.708610,-0.460745,1.0,2.0,19.000000,1
750,-1.094881,-0.646436,-1.102782,-1.123805,-0.068725,-0.132396,0.382150,-0.145487,-1.271607,0.491977,...,-1.926701,0.030861,1.267376,0,-1.764768,-0.848211,1.0,2.0,19.000000,1


In [12]:
# 필요한 열을 선택하여 x와 y 생성
X = result_nona[['EDA_mean']]
Y = result_nona[['call_label']]

# 결과 확인
print("X (data):")
print(X)

print("\nY (label):")
print(Y)

X (data):
     EDA_mean
0   -3.134496
1   -3.132410
2   -3.134362
3   -3.127229
4   -3.107917
..        ...
747 -1.225691
748 -1.185856
749 -1.138687
750 -1.094881
751 -1.061977

[752 rows x 1 columns]

Y (label):
     call_label
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
..          ...
747         1.0
748         1.0
749         1.0
750         1.0
751         1.0

[752 rows x 1 columns]


In [13]:
import torch
from MKLpy.preprocessing import \
    normalization, rescale_01, rescale, centering
from MKLpy.utils.validation import check_X

X_numpy = X.values  # Convert DataFrame to a NumPy array
X_tensor = torch.tensor(X_numpy, dtype=torch.float32) #numpy -> tensor 
#X_tensor = check_X(X_tensor)

X = rescale_01(X_tensor)
print(X)

tensor([[8.7317e-03],
        [9.0922e-03],
        [8.7550e-03],
        [9.9874e-03],
        [1.3324e-02],
        [1.6987e-02],
        [1.9986e-02],
        [2.1870e-02],
        [2.2010e-02],
        [1.9870e-02],
        [1.7545e-02],
        [1.6359e-02],
        [1.5080e-02],
        [1.3103e-02],
        [9.7316e-03],
        [6.2087e-03],
        [3.1276e-03],
        [9.7664e-04],
        [0.0000e+00],
        [2.8718e-03],
        [7.7435e-03],
        [1.3010e-02],
        [2.0068e-02],
        [2.7265e-02],
        [3.3660e-02],
        [3.9961e-02],
        [4.6100e-02],
        [5.1484e-02],
        [5.6483e-02],
        [6.0494e-02],
        [6.5471e-02],
        [7.5484e-02],
        [8.6521e-02],
        [9.5034e-02],
        [1.0518e-01],
        [1.1763e-01],
        [1.3081e-01],
        [1.4312e-01],
        [1.5243e-01],
        [1.5788e-01],
        [1.5938e-01],
        [1.5727e-01],
        [1.6222e-01],
        [1.7250e-01],
        [1.7931e-01],
        [1

In [14]:
from MKLpy.metrics.pairwise import homogeneous_polynomial_kernel as hpk
K_train = hpk(X, degree=2)
#K_test  = hpk(X, degree=2)
print(K_train)

tensor([[5.8130e-09, 6.3028e-09, 5.8440e-09,  ..., 9.5321e-06, 9.9446e-06,
         1.0260e-05],
        [6.3028e-09, 6.8340e-09, 6.3365e-09,  ..., 1.0335e-05, 1.0783e-05,
         1.1125e-05],
        [5.8440e-09, 6.3365e-09, 5.8752e-09,  ..., 9.5830e-06, 9.9977e-06,
         1.0315e-05],
        ...,
        [9.5321e-06, 1.0335e-05, 9.5830e-06,  ..., 1.5631e-02, 1.6307e-02,
         1.6825e-02],
        [9.9446e-06, 1.0783e-05, 9.9977e-06,  ..., 1.6307e-02, 1.7013e-02,
         1.7553e-02],
        [1.0260e-05, 1.1125e-05, 1.0315e-05,  ..., 1.6825e-02, 1.7553e-02,
         1.8110e-02]], dtype=torch.float64)


In [15]:
from MKLpy.generators import HPK_generator
KL = HPK_generator(X, degrees=range(1,11), cache=True)
print(KL)

In [16]:
print(Y)

     call_label
0           0.0
1           0.0
2           0.0
3           0.0
4           0.0
..          ...
747         1.0
748         1.0
749         1.0
750         1.0
751         1.0

[752 rows x 1 columns]


In [17]:
import numpy as np
Y = Y.to_numpy()
Y = Y.reshape(-1)

print(Y)
# train_test_split 함수를 사용하여 데이터 분할
from MKLpy.model_selection import train_test_split

# train 및 test 변수 정의
KLtr, KLte, Ytr, Yte = train_test_split(KL, Y, test_size=.3, random_state=42)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.

In [18]:
print(KLtr, KLte, Ytr, Yte)

[tensor([[0.5508, 0.5978, 0.4920,  ..., 0.5409, 0.4937, 0.4626],
        [0.5978, 0.6489, 0.5340,  ..., 0.5871, 0.5358, 0.5021],
        [0.4920, 0.5340, 0.4394,  ..., 0.4831, 0.4410, 0.4132],
        ...,
        [0.5409, 0.5871, 0.4831,  ..., 0.5312, 0.4848, 0.4543],
        [0.4937, 0.5358, 0.4410,  ..., 0.4848, 0.4425, 0.4146],
        [0.4626, 0.5021, 0.4132,  ..., 0.4543, 0.4146, 0.3885]],
       dtype=torch.float64), tensor([[0.3034, 0.3574, 0.2421,  ..., 0.2926, 0.2437, 0.2140],
        [0.3574, 0.4210, 0.2851,  ..., 0.3447, 0.2871, 0.2521],
        [0.2421, 0.2851, 0.1931,  ..., 0.2334, 0.1945, 0.1707],
        ...,
        [0.2926, 0.3447, 0.2334,  ..., 0.2821, 0.2350, 0.2064],
        [0.2437, 0.2871, 0.1945,  ..., 0.2350, 0.1958, 0.1719],
        [0.2140, 0.2521, 0.1707,  ..., 0.2064, 0.1719, 0.1509]],
       dtype=torch.float64), tensor([[0.1671, 0.2137, 0.1191,  ..., 0.1583, 0.1203, 0.0990],
        [0.2137, 0.2732, 0.1523,  ..., 0.2023, 0.1539, 0.1266],
        [0.1191, 

In [19]:
from sklearn.svm import SVC
base_learner = SVC(C=100)

# from MKLpy.algorithms import EasyMKL
# mkl = EasyMKL(lam=0.1, learner=base_learner)
# mkl = mkl.fit(KLtr, Ytr)

from MKLpy.algorithms import AverageMKL
#mkl = AverageMKL(learner=base_learner).fit(KLtr, Ytr) #base_learner에 svm, 로지스틱 회귀 (learner 명시적 선)
#mkl = AverageMKL(multiclass_strategy='ova').fit(KLtr, Ytr) #지정된 멀티클래스 전략을 사용(여러 클래스를 처리하기 위한)
mkl = AverageMKL().fit(KLtr, Ytr)       #combine kernels and train the classifier (기본설정)

In [21]:
y_preds  = mkl.predict(KLte)            #predict the output class
y_scores = mkl.decision_function(KLte)  #returns the projection on the distance vector

In [22]:
print(y_preds)
print(y_scores)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[ 0.99173413  1.02973528  0.98932165  1.03263236  0.97556051  1.46819567
  1.02664784  0.98579576  0.96197149  0.99374025  0.98360369  0.98918354
  1.00860914  1.00579722  1.03258383  0.97019832  1.00492496  1.00248403
  1.00950186  0.99647206  0.98521892  1.42521655  0.96536904  0.9763556
  0.97804262  0.0254

In [27]:
#evaluation with scikit-learn metrics
from sklearn.metrics import accuracy_score, roc_auc_score
accuracy = accuracy_score(Yte, y_preds)
roc_auc = roc_auc_score(Yte, y_scores)
print ('Accuracy score: %.4f, roc AUC score: %.4f' % (accuracy, roc_auc))

Accuracy score: 0.6770, roc AUC score: 0.5650
